<a href="https://colab.research.google.com/github/christophermalone/DSCI415/blob/main/NLP_StudentLoanReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task #6:  Natural Language Processing - Student Loan Complaints

For this task, you are asked to visit the Consumer Finanical Protection Bureau website. This organization maintains a Customer Complaint Database where customers are allowed to file complaints finanical institutions. 

Customer Complaint Database:  https://www.consumerfinance.gov/data-research/consumer-complaints/search  

Visit the Customer Complaint Database website.  Apply the following two filters, and then download the data.

1.   Only obtain complaints that have been filed in 2021
2.   Only obtain complaints that are for Product / sub product = Student loan



After downloading the desired complaint data, upload the *.csv file into the sample_data folder in Google Colab.  The following Python code can be used to read in the desired complaint data.

In [74]:
# install and import pandas library
import pandas as pd

# Creating a pandas dataframe from reviews.txt file
Complaints = pd.read_csv('/content/sample_data/StudentLoan_Complaints2021.csv', sep=',')

#Complaints.shape
Complaints.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,05/04/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Problem with customer service,I have made several payments that have been de...,None,AES/PHEAA,VA,22201,None,Consent provided,Web,05/04/21,Closed with non-monetary relief,Yes,NaN,4347871
1,06/24/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Keep getting calls about your loan,Hello - I received a predatory call from Great...,Company believes it acted appropriately as aut...,"Nelnet, Inc.",OH,44092,Servicemember,Consent provided,Web,06/24/21,Closed with explanation,Yes,NaN,4488517
2,07/01/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Received bad information about your loan,I applied for an income-driven repayment plan ...,None,AES/PHEAA,CA,94110,None,Consent provided,Web,07/01/21,Closed with explanation,Yes,NaN,4509407
3,07/14/21,Student loan,Private student loan,Incorrect information on your report,Account status incorrect,I dispute this information with Navient and th...,None,"Navient Solutions, LLC.",VA,23238,None,Consent provided,Web,07/14/21,Closed with explanation,Yes,NaN,4542318
4,03/04/21,Student loan,Private student loan,Dealing with your lender or servicer,Need information about your loan balance or lo...,NaN,None,"CITIZENS FINANCIAL GROUP, INC.",KY,40505,None,NaN,Referral,03/07/21,Closed with explanation,Yes,NaN,4187037


In [83]:
Complaints = Complaints.dropna(subset=['Consumer complaint narrative'])

#Complaints.shape
Complaints.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,OnlyCharacters_Narrative,Cleaned_Narrative_POS
0,05/04/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Problem with customer service,I have made several payments that have been de...,None,AES/PHEAA,VA,22201,None,Consent provided,Web,05/04/21,Closed with non-monetary relief,Yes,NaN,4347871,I have made several payments that have been de...,"[(made, v), (several, a), (payments, n), (deem..."
1,06/24/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Keep getting calls about your loan,Hello - I received a predatory call from Great...,Company believes it acted appropriately as aut...,"Nelnet, Inc.",OH,44092,Servicemember,Consent provided,Web,06/24/21,Closed with explanation,Yes,NaN,4488517,Hello I received a predatory call from Great L...,"[(Hello, n), (received, v), (predatory, n), (c..."
2,07/01/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Received bad information about your loan,I applied for an income-driven repayment plan ...,None,AES/PHEAA,CA,94110,None,Consent provided,Web,07/01/21,Closed with explanation,Yes,NaN,4509407,I applied for an income driven repayment plan ...,"[(applied, v), (income, n), (driven, v), (repa..."
3,07/14/21,Student loan,Private student loan,Incorrect information on your report,Account status incorrect,I dispute this information with Navient and th...,None,"Navient Solutions, LLC.",VA,23238,None,Consent provided,Web,07/14/21,Closed with explanation,Yes,NaN,4542318,I dispute this information with Navient and th...,"[(dispute, v), (information, n), (Navient, n),..."
5,06/30/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,I submitted my employer certification form for...,None,AES/PHEAA,PA,18902,None,Consent provided,Web,06/30/21,Closed with non-monetary relief,Yes,NaN,4504253,I submitted my employer certification form for...,"[(submitted, v), (employer, n), (certification..."


In [84]:
#Importing the regular experssions library
import re

# Define a function to clean the text
def clean(text):
    # Removes all special characters and numericals leaving the alphabets
    text = re.sub('[^A-Za-z]+', ' ', str(text))
    text = re.sub(r'XX\S+', '', str(text))  # remove redacted words
    return text

In [85]:
# Cleaning the text in the review column
Complaints['OnlyCharacters_Narrative'] = Complaints['Consumer complaint narrative'].apply(clean)
Complaints.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,OnlyCharacters_Narrative,Cleaned_Narrative_POS
0,05/04/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Problem with customer service,I have made several payments that have been de...,None,AES/PHEAA,VA,22201,None,Consent provided,Web,05/04/21,Closed with non-monetary relief,Yes,NaN,4347871,I have made several payments that have been de...,"[(made, v), (several, a), (payments, n), (deem..."
1,06/24/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Keep getting calls about your loan,Hello - I received a predatory call from Great...,Company believes it acted appropriately as aut...,"Nelnet, Inc.",OH,44092,Servicemember,Consent provided,Web,06/24/21,Closed with explanation,Yes,NaN,4488517,Hello I received a predatory call from Great L...,"[(Hello, n), (received, v), (predatory, n), (c..."
2,07/01/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Received bad information about your loan,I applied for an income-driven repayment plan ...,None,AES/PHEAA,CA,94110,None,Consent provided,Web,07/01/21,Closed with explanation,Yes,NaN,4509407,I applied for an income driven repayment plan ...,"[(applied, v), (income, n), (driven, v), (repa..."
3,07/14/21,Student loan,Private student loan,Incorrect information on your report,Account status incorrect,I dispute this information with Navient and th...,None,"Navient Solutions, LLC.",VA,23238,None,Consent provided,Web,07/14/21,Closed with explanation,Yes,NaN,4542318,I dispute this information with Navient and th...,"[(dispute, v), (information, n), (Navient, n),..."
5,06/30/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,I submitted my employer certification form for...,None,AES/PHEAA,PA,18902,None,Consent provided,Web,06/30/21,Closed with non-monetary relief,Yes,NaN,4504253,I submitted my employer certification form for...,"[(submitted, v), (employer, n), (certification..."


In [86]:
print('\n')
print(Complaints.iloc[1]['Consumer complaint narrative'])

print('\n')
print(Complaints.iloc[1]['OnlyCharacters_Narrative'])




Hello - I received a predatory call from Great Lakes Educational Loan Services , Inc. where they posed as my student loan servicer and received my personal information before I figured out they were not who the actual loan company. The agent was quite rude when I questioned his intent and got verbally abusive on the phone. Now, after this contact I'm concerned that they may use the information I provided for means that I do not approve. 

I do not appreciate their deceptive rude tactic and would someone to reprimand them for these practices. 

This practice of calling a list of people with student loans is very tricky and disturbing. 

Company info based on the phone number XXXX search match. 
Great Lakes Educational Loan Services , Inc .


Hello I received a predatory call from Great Lakes Educational Loan Services Inc where they posed as my student loan servicer and received my personal information before I figured out they were not who the actual loan company The agent was quite r

In [87]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet

nltk.download('averaged_perceptron_tagger')

# POS tagger dictionary
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}

def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
            newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [88]:
Complaints['Cleaned_Narrative_POS'] = Complaints['OnlyCharacters_Narrative'].apply(token_stop_pos)
Complaints.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,OnlyCharacters_Narrative,Cleaned_Narrative_POS
0,05/04/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Problem with customer service,I have made several payments that have been de...,None,AES/PHEAA,VA,22201,None,Consent provided,Web,05/04/21,Closed with non-monetary relief,Yes,NaN,4347871,I have made several payments that have been de...,"[(made, v), (several, a), (payments, n), (deem..."
1,06/24/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Keep getting calls about your loan,Hello - I received a predatory call from Great...,Company believes it acted appropriately as aut...,"Nelnet, Inc.",OH,44092,Servicemember,Consent provided,Web,06/24/21,Closed with explanation,Yes,NaN,4488517,Hello I received a predatory call from Great L...,"[(Hello, n), (received, v), (predatory, n), (c..."
2,07/01/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Received bad information about your loan,I applied for an income-driven repayment plan ...,None,AES/PHEAA,CA,94110,None,Consent provided,Web,07/01/21,Closed with explanation,Yes,NaN,4509407,I applied for an income driven repayment plan ...,"[(applied, v), (income, n), (driven, v), (repa..."
3,07/14/21,Student loan,Private student loan,Incorrect information on your report,Account status incorrect,I dispute this information with Navient and th...,None,"Navient Solutions, LLC.",VA,23238,None,Consent provided,Web,07/14/21,Closed with explanation,Yes,NaN,4542318,I dispute this information with Navient and th...,"[(dispute, v), (information, n), (Navient, n),..."
5,06/30/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,I submitted my employer certification form for...,None,AES/PHEAA,PA,18902,None,Consent provided,Web,06/30/21,Closed with non-monetary relief,Yes,NaN,4504253,I submitted my employer certification form for...,"[(submitted, v), (employer, n), (certification..."


In [89]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos: 
            lemma = word
            lemma_rew = lemma_rew + " " + lemma
        else:  
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
            lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

In [90]:
Complaints['Cleaned_Narrative'] = Complaints['Cleaned_Narrative_POS'].apply(lemmatize)
Complaints.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,OnlyCharacters_Narrative,Cleaned_Narrative_POS,Cleaned_Narrative
0,05/04/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Problem with customer service,I have made several payments that have been de...,None,AES/PHEAA,VA,22201,None,Consent provided,Web,05/04/21,Closed with non-monetary relief,Yes,NaN,4347871,I have made several payments that have been de...,"[(made, v), (several, a), (payments, n), (deem...",make several payment deem ineligible towards...
1,06/24/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Keep getting calls about your loan,Hello - I received a predatory call from Great...,Company believes it acted appropriately as aut...,"Nelnet, Inc.",OH,44092,Servicemember,Consent provided,Web,06/24/21,Closed with explanation,Yes,NaN,4488517,Hello I received a predatory call from Great L...,"[(Hello, n), (received, v), (predatory, n), (c...",Hello receive predatory call Great Lakes Edu...
2,07/01/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Received bad information about your loan,I applied for an income-driven repayment plan ...,None,AES/PHEAA,CA,94110,None,Consent provided,Web,07/01/21,Closed with explanation,Yes,NaN,4509407,I applied for an income driven repayment plan ...,"[(applied, v), (income, n), (driven, v), (repa...",apply income drive repayment plan work towar...
3,07/14/21,Student loan,Private student loan,Incorrect information on your report,Account status incorrect,I dispute this information with Navient and th...,None,"Navient Solutions, LLC.",VA,23238,None,Consent provided,Web,07/14/21,Closed with explanation,Yes,NaN,4542318,I dispute this information with Navient and th...,"[(dispute, v), (information, n), (Navient, n),...",dispute information Navient keep parrot back...
5,06/30/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,I submitted my employer certification form for...,None,AES/PHEAA,PA,18902,None,Consent provided,Web,06/30/21,Closed with non-monetary relief,Yes,NaN,4504253,I submitted my employer certification form for...,"[(submitted, v), (employer, n), (certification...",submit employer certification form Public Se...


Reduce the Complaints data.frame down to only needed columns.

In [132]:
Complaints_Analysis = pd.DataFrame(Complaints[['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue', 'Consumer complaint narrative', 'Company', 'Cleaned_Narrative']])
Complaints_Analysis.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company,Cleaned_Narrative
0,05/04/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Problem with customer service,I have made several payments that have been de...,AES/PHEAA,make several payment deem ineligible towards...
1,06/24/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Keep getting calls about your loan,Hello - I received a predatory call from Great...,"Nelnet, Inc.",Hello receive predatory call Great Lakes Edu...
2,07/01/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Received bad information about your loan,I applied for an income-driven repayment plan ...,AES/PHEAA,apply income drive repayment plan work towar...
3,07/14/21,Student loan,Private student loan,Incorrect information on your report,Account status incorrect,I dispute this information with Navient and th...,"Navient Solutions, LLC.",dispute information Navient keep parrot back...
5,06/30/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,I submitted my employer certification form for...,AES/PHEAA,submit employer certification form Public Se...


In [133]:
from textblob import TextBlob

# function to calculate subjectivity 
def getSubjectivity(review):
    return TextBlob(review).sentiment.subjectivity

# function to calculate polarity
def getPolarity(review):
    return TextBlob(review).sentiment.polarity


In [134]:
#Complaints_Analysis['Subjectivity'] = Complaints_Analysis['Cleaned_Narrative'].apply(getSubjectivity) 
Complaints_Analysis['Polarity'] = Complaints_Analysis['Cleaned_Narrative'].apply(getPolarity) 
Complaints_Analysis.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company,Cleaned_Narrative,Polarity
0,05/04/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Problem with customer service,I have made several payments that have been de...,AES/PHEAA,make several payment deem ineligible towards...,-0.109470
1,06/24/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Keep getting calls about your loan,Hello - I received a predatory call from Great...,"Nelnet, Inc.",Hello receive predatory call Great Lakes Edu...,0.038750
2,07/01/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Received bad information about your loan,I applied for an income-driven repayment plan ...,AES/PHEAA,apply income drive repayment plan work towar...,0.067273
3,07/14/21,Student loan,Private student loan,Incorrect information on your report,Account status incorrect,I dispute this information with Navient and th...,"Navient Solutions, LLC.",dispute information Navient keep parrot back...,-0.093750
5,06/30/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,I submitted my employer certification form for...,AES/PHEAA,submit employer certification form Public Se...,0.069464


In [135]:
#Install dfply for summaries, etc
!pip install dfply
from dfply import *

In [157]:
#Getting a summary for Polarity
Summarize = (
             Complaints_Analysis
             >> group_by(X.Issue)
             >> summarize(Avg_Polarity=X.Polarity.mean())
             >> ungroup()
             >> arrange(X.Avg_Polarity)
            )
Summarize


,Issue,Avg_Polarity
5,Problem with a credit reporting company's inve...,-0.012177
6,Problem with fraud alerts or security freezes,-0.010877
4,Incorrect information on your report,0.000680
7,Struggling to repay your loan,0.001413
3,Improper use of your report,0.002175
1,Dealing with your lender or servicer,0.014273
2,Getting a loan,0.024437
0,Credit monitoring or identity theft protection...,0.025381


In [137]:
#Install package using pip
!pip install text2emotion

#Import the modules
import text2emotion as te

# function to calculate polarity
def getEmotion(review):
    return te.get_emotion(review)

     |████████████████████████████████| 57 kB 2.4 MB/s 
     |████████████████████████████████| 170 kB 7.8 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=354901ddaea7092055403be57247f520ab7f801a5b0687da18350faa3105f5ff
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [138]:
#Get the emotion scores
#BE CAREFUL THIS CELL TAKES SEVERAL MINUTES TO RUN
Complaints_Analysis['Emotion'] = Complaints_Analysis['Cleaned_Narrative'].apply(getEmotion)


In [151]:
#Split emotion list up into seperate columns
Complaints_Analysis[['Happy','Angry', 'Surprise', 'Sad', 'Fear']] = pd.DataFrame(Complaints_Analysis.Emotion.values.tolist(), index= Complaints_Analysis.index)

Complaints_Analysis.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company,Cleaned_Narrative,Polarity,Emotion,Happy,Angry,Surprise,Sad,Fear
0,05/04/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Problem with customer service,I have made several payments that have been de...,AES/PHEAA,make several payment deem ineligible towards...,-0.109470,"{'Happy': 0.09, 'Angry': 0.0, 'Surprise': 0.18...",0.09,0.00,0.18,0.27,0.45
1,06/24/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Keep getting calls about your loan,Hello - I received a predatory call from Great...,"Nelnet, Inc.",Hello receive predatory call Great Lakes Edu...,0.038750,"{'Happy': 0.24, 'Angry': 0.0, 'Surprise': 0.29...",0.24,0.00,0.29,0.12,0.35
2,07/01/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Received bad information about your loan,I applied for an income-driven repayment plan ...,AES/PHEAA,apply income drive repayment plan work towar...,0.067273,"{'Happy': 0.11, 'Angry': 0.0, 'Surprise': 0.11...",0.11,0.00,0.11,0.22,0.56
3,07/14/21,Student loan,Private student loan,Incorrect information on your report,Account status incorrect,I dispute this information with Navient and th...,"Navient Solutions, LLC.",dispute information Navient keep parrot back...,-0.093750,"{'Happy': 0.2, 'Angry': 0.0, 'Surprise': 0.0, ...",0.20,0.00,0.00,0.30,0.50
5,06/30/21,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,I submitted my employer certification form for...,AES/PHEAA,submit employer certification form Public Se...,0.069464,"{'Happy': 0.16, 'Angry': 0.04, 'Surprise': 0.2...",0.16,0.04,0.28,0.20,0.32


A summary of angry...

In [153]:
#Getting a summary for Angry
Summarize = (
             Complaints_Analysis
             >> group_by(X.Company)
             >> summarize( Avg_Angry=X.Angry.mean()
                          )
             >> ungroup()
             >> arrange(X.Avg_Angry, ascending = False)
            )
Summarize.head(10)

,Company,Avg_Angry
30,Direct Account Management,0.150000
98,Students Come First,0.145000
19,College Assist,0.140000
11,BCO Consulting Services,0.113333
72,New Hampshire Higher Education Loan Corporation,0.110000
14,"CITIBANK, N.A.",0.110000
31,DocuPrep Xpress,0.103333
26,Credit Adjustments Inc,0.100000
69,National Student Debt Advisory Corp.,0.100000
78,PNC Bank N.A.,0.090000
